In [41]:
import importlib 

import ball
from ball import Ball
importlib.reload(ball)

import scipy.stats as stats
from scipy.stats import maxwell

import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation

from IPython.display import HTML

In [42]:
# num_frames = int(input("Enter the # of frames:"))
num_frames = 3000

# r = int(input("Enter rows:"))
# c = int(input("Enter columns:"))
x = 9
y = 9

n = [x, y]
nx = n[0]
ny = n[1]

balls = []
adjacent_cells = []

# velocity_multiplier = float(input("Enter the velocity multiplyer:"))
velocity_multiplier = 1.5

In [43]:
# set up the figure and axis
plt.ioff() # stop automatic inline display
fig_gas, gas = plt.subplots(figsize=(5, 5))
fig_graph, graph = plt.subplots(figsize=(5, 5))

gas.set_xlim(0, n[0])
gas.set_ylim(0, n[1])
gas.set_aspect("equal")

# obtain positions for nxn grid
for i in range(nx):
    for j in range(ny):
        # create a dot at an initial position
        circle = patches.Circle((i + 0.5, j + 0.5), radius=0.05, facecolor="blue")
        gas.add_patch(circle)
        b = Ball([abs(i + 0.5), abs(j + 0.5)], circle)
        balls.append(b)

In [44]:
def plot_boltzmann(velocities):
    graph.clear()
    graph.set_ylim(0, 60)
    graph.set_xlim(0, 0.055)
    graph.set_xlabel("Velocities")
    graph.set_ylabel("Frequency")
    graph.set_title('Maxwell-Boltzmann distribution: # of balls vs. velocity')

    speeds = [np.linalg.norm(v) for v in velocities]
    
    bin_width = 0.005
    min_val = min(speeds)
    max_val = max(speeds)
    bins = np.arange(0, max_val + bin_width, bin_width)

    graph.hist(speeds, bins=bins, edgecolor='black')

    x_vals = np.linspace(0, 0.075, 100)
    graph.plot(x_vals, maxwell.pdf(100 * x_vals) * 45)

In [45]:
def calculate_collision(v1, v2, x1, x2):
    new_v1 = v1 - ((np.dot(v1 - v2, x1 - x2) /
                    np.dot(x1 - x2, x1 - x2))       * (x1 - x2))
    new_v2 = v2 - ((np.dot(v2 - v1, x2 - x1) /
                    np.dot(x2 - x1, x2 - x1))       * (x2 - x1))

    if np.allclose(x1, x2):
        print("Collision skipped: identical positions")
        return v1, v2
    else:
        return new_v1, new_v2

In [46]:
def check_collision(v1, x1, adjacent_cells, colliding, ball):
    adjacent_cells_balls = []

    # find balls in adjacent cells
    for next_ball in balls:
        next_x = next_ball.position
        next_cell = [int(next_x[0]), int(next_x[1])]

        if next_cell in adjacent_cells and next_ball != ball:
            adjacent_cells_balls.append(next_ball)
    
    # check for collisions with balls in adjacent cells
    for adj_ball in adjacent_cells_balls:
        x2 = adj_ball.position
        v2 = adj_ball.velocity
        circle2 = adj_ball.circle
        colliding2 = adj_ball.colliding

        distance = math.sqrt(((x2[0] - x1[0]) ** 2) + ((x2[1] - x1[1]) ** 2))

        if distance <= 0.1 and not colliding and not colliding2:
            # print("collision detected")
            colliding = True
            
        elif distance > 0.1 and colliding:
            colliding = False

        if colliding:
            v1, v2 = calculate_collision(v1, v2, x1, x2)

            x2 += v2 * velocity_multiplier
            circle2.center = (x2[0], x2[1])

            adj_ball.position = x2
            adj_ball.velocity = v2
            adj_ball.circle = circle2
            adj_ball.colliding = colliding2

            colliding = False


    return v1

In [47]:
# animation update function
def update(frame):
    velocities = []
    for ball in balls:
        # --- update attributes ---
        x1 = ball.position
        v1 = ball.velocity
        nv1 = v1
        circle = ball.circle
        colliding = ball.colliding

        cell = [int(x1[0]), int(x1[1])] # each cell is 1 x 1
        cell_x = cell[0]
        cell_y = cell[1]

        # --- neigboring cells ---
        # - corners -
        if cell_x == 0 and cell_y == 0:                 # LEFT top
            adjacent_cells = [[cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_x == nx and cell_y == 0:              # RIGHT top
            adjacent_cells = [[cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y]]
        elif cell_x == nx and cell_y == ny:             # RIGHT top
            adjacent_cells = [[cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1],
                              [cell_x, cell_y - 1]]
        elif cell_x == 0 and cell_y == ny:              # LEFT bottom
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y]]

        # - edges -
        elif cell_x == 0:                               # LEFT
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_y == 0:                               # TOP
            adjacent_cells = [[cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y]]
        elif cell_x == nx:                              # RIGHT
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x - 1, cell_y - 1],
                              [cell_x - 1, cell_y],
                              [cell_x - 1, cell_y + 1],
                              [cell_x, cell_y + 1]]
        elif cell_y == ny:                              # BOTTOM
            adjacent_cells = [[cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1],
                              [cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y]]

        # - center -
        else:
            adjacent_cells = [[cell_x, cell_y - 1],
                              [cell_x + 1, cell_y - 1],
                              [cell_x + 1, cell_y],
                              [cell_x + 1, cell_y + 1],
                              [cell_x, cell_y + 1],
                              [cell_x - 1, cell_y + 1],
                              [cell_x - 1, cell_y],
                              [cell_x - 1, cell_y - 1]]

        adjacent_cells.append(cell) # include current cell

        nv1 = check_collision(v1, x1, adjacent_cells, colliding, ball)

        # wall collision: check if ball is outside bounds of nxn grid and reverse direction of velocity
        if x1[0] >= (n[0]):
            nv1[0] = -abs(nv1[0])
            x1 += nv1
        elif x1[0] <= 0:
            nv1[0] = abs(nv1[0])
            x1 += nv1

        if x1[1] >= (n[1]):
            nv1[1] = -abs(nv1[1])
            x1 += nv1
        elif x1[1] <= 0:
            nv1[1] = abs(v1[1])
            x1 += nv1

        x1 += nv1 * velocity_multiplier

        circle.center = (x1[0], x1[1])

        # update inherent ball position
        ball.position = x1
        ball.velocity = nv1 
        ball.circle = circle
        ball.colliding = colliding

        velocities.append(nv1)
    
    # plot_boltzmann(velocities)

In [48]:

# animation in notebook
# anim = animation.FuncAnimation(fig_graph, update, frames=num_frames, interval=1)
# matplotlib.rcParams['animation.embed_limit'] = 2**128
# HTML(animation.to_jshtml()) # render inline

# write to file
anim = animation.FuncAnimation(fig_gas, update, frames=num_frames, interval=1)
writergif = animation.PillowWriter(fps=30)
anim.save("renders/boltzmann-distribution-ideal-gas-simulation.gif", writer=writergif)